In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from pathlib import Path
from typing import List
from optimum.pipelines import pipeline
from optimum.onnxruntime import AutoOptimizationConfig, ORTModelForFeatureExtraction, ORTOptimizer
from optimum.onnxruntime.configuration import OptimizationConfig

import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel

In [8]:
# Load the tokenizer and export the model to the ONNX format
# model_id = "sentence-transformers/all-MiniLM-L6-v2"
# model_id = "thenlper/gte-base"
# model_id = "intfloat/multilingual-e5-large"
model_id = "BAAI/bge-base-en"
save_dir = f"fast-{model_id.split('/')[1]}"
print(save_dir)

fast-bge-base-en


In [9]:
def average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.
input_texts = [
    "query: how much protein should a female eat",
    "query: 南瓜的家常做法",
    "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
    "passage: 1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅",
]

hf_model = AutoModel.from_pretrained(model_id)
hf_tokenizer = AutoTokenizer.from_pretrained(model_id)


def hf_embed(model_id: str, input_text: List[str]):
    # Tokenize the input texts
    batch_dict = hf_tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors="pt")

    outputs = hf_model(**batch_dict)
    embeddings = average_pool(outputs.last_hidden_state, batch_dict["attention_mask"])

    # normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)
    scores = (embeddings[:2] @ embeddings[2:].T) * 100
    # print(scores.tolist())
    return scores

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = ORTModelForFeatureExtraction.from_pretrained(model_id, export=True)

# Remove all existing files in the save_dir using Path.unlink()
save_dir = Path(save_dir)
save_dir.mkdir(parents=True, exist_ok=True)
for p in save_dir.iterdir():
    p.unlink()

# Load the optimization configuration detailing the optimization we wish to apply
optimization_config = AutoOptimizationConfig.O3()
optimizer = ORTOptimizer.from_pretrained(model)

optimizer.optimize(save_dir=save_dir, optimization_config=optimization_config, use_external_data_format=True)
model = ORTModelForFeatureExtraction.from_pretrained(save_dir)

tokenizer.save_pretrained(save_dir)
# model.save_pretrained(save_dir)
# model.push_to_hub("new_path_for_directory", repository_id="my-onnx-repo", use_auth_token=True)

Framework not specified. Using pt to export to ONNX.
Using framework PyTorch: 2.0.1
Overriding 1 configuration item(s)
	- use_cache -> False


================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



The argument use_external_data_format in the ORTOptimizer.optimize() method is deprecated and will be removed in optimum 2.0.
Optimizing model...
Configuration saved in fast-bge-base-en/ort_config.json
Optimized model saved at: fast-bge-base-en (external data format: False; saved all tensor to one file: True)


('fast-bge-base-en/tokenizer_config.json',
 'fast-bge-base-en/special_tokens_map.json',
 'fast-bge-base-en/vocab.txt',
 'fast-bge-base-en/added_tokens.json',
 'fast-bge-base-en/tokenizer.json')

In [11]:
# onnx_embed = pipeline("feature-extraction", model="sentence-transformers/all-MiniLM-L6-v2", accelerator="ort")
question = "What's my name?" * 50

In [12]:
onnx_quant_embed = pipeline("feature-extraction", model=model, accelerator="ort")

# Ours

In [13]:
%%timeit
pred = onnx_quant_embed(question)

105 ms ± 237 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Original

In [14]:
%%timeit
embeddings = hf_embed(model_id, question)

276 ms ± 5.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Compress & Upload

## Compress

In [15]:
import os
from pathlib import Path
import tarfile


def compress(directory_path):
    directory_path = Path(directory_path)
    assert directory_path.exists(), f"{directory_path} does not exist"
    output_filename = directory_path.name + ".tar.gz"
    if Path(output_filename).exists():
        print("We've an output file already? Manually delete that first")
        return output_filename

    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(directory_path, arcname=os.path.basename(directory_path))
    return output_filename


compressed_file_name = compress(save_dir)

## Upload to Qdrant Google Cloud Storage

In [16]:
from google.cloud import storage


def upload(bucket_name, source_file_path):
    storage_client = storage.Client(project="main")
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(os.path.basename(source_file_path))

    blob.upload_from_filename(source_file_path)

    print(f"File {source_file_path} uploaded to {bucket_name}.")


upload("qdrant-fastembed", compressed_file_name)

ModuleNotFoundError: No module named 'google.cloud'

In [ ]:
# Remove the directory and compressed file
!rm -rvf {save_dir}
!rm -vf {save_dir}.tar.gz